In [1]:
import torch
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import torch.nn as nn
import pandas as pd
import numpy as np


if torch.cuda.is_available():
    device=torch.device(type="cuda",index=0)
else:
    device=torch.device(type="cpu",index=0)

print(device)

class TrainDS(Dataset):
    def __init__(self, path, transform):
        super().__init__()
        self.data=pd.read_csv(path,header='infer').values
        self.length=self.data.shape[0]
        self.transform=transform
    
    def __len__(self):
        return self.length
    
    def __getitem__(self,idx):
        flatimage=self.data[idx,1:].astype(np.uint8)
        image=self.transform(np.reshape(flatimage,(28,28,1)))
        label=self.data[idx,0]
        return image,label

class TestDS(Dataset):
    def __init__(self, path, transform):
        super().__init__()
        self.data=pd.read_csv(path,header='infer').values
        self.length=self.data.shape[0]
        self.transform=transform
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        flatimage=self.data[idx,:].astype(np.uint8)
        image=self.transform(np.reshape(flatimage,(28,28,1)))
        return image
    
trainDS=TrainDS('/kaggle/input/digit-recognizer/train.csv', ToTensor())
testDS=TestDS('/kaggle/input/digit-recognizer/test.csv', ToTensor())

batch_size=64

trainDL=DataLoader(dataset=trainDS, batch_size=batch_size, shuffle=True)
testDL=DataLoader(dataset=testDS, batch_size=batch_size)

class prac4(nn.Module):
    def __init__(self):
        super().__init__()
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=8, kernel_size=(3,3), stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(8)
        self.mp1 = nn.MaxPool2d(kernel_size=(2,2),stride=2,padding=0)
        self.conv2 = nn.Conv2d(in_channels=8,out_channels=16, kernel_size=(3,3), stride=1, padding=0)
        self.bn2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(in_channels=16,out_channels=32, kernel_size=(3,3), stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(in_channels=32,out_channels=64, kernel_size=(3,3), stride=1, padding=0)
        self.bn4 = nn.BatchNorm2d(64)
        self.flatten = nn.Flatten()
        self.full = nn.Linear(in_features=3136, out_features=10)
        self.bn5 = nn.BatchNorm1d(num_features=10)
            
    def forward(self,x):
        x=self.conv1(x)
        x=self.bn1(x)
        x=self.relu(x)
        x=self.mp1(x)
        x=self.conv2(x)
        x=self.bn2(x)
        x=self.relu(x)
        x=self.conv3(x)
        x=self.bn3(x)
        x=self.relu(x)
        x=self.conv4(x)
        x=self.bn4(x)
        x=self.relu(x)
        x=self.flatten(x)
        x=self.full(x)
        output=self.bn5(x)
        
        return output
    
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    track_loss = 0
    num_correct = 0
    for i, (imgs, labels) in enumerate(dataloader):
        imgs = imgs.to(device)
        labels = labels.to(device)
        pred = model(imgs)
                    
        loss = loss_fn(pred, labels)
        track_loss += loss.item()
        num_correct += (torch.argmax(pred, dim=1)==labels).type(torch.float).sum().item()
        
        running_loss = round(track_loss/(i+(imgs.shape[0]/batch_size)),2)
        running_acc = round((num_correct/((i*batch_size+imgs.shape[0])))*100,2)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i%100==0:
            print("Batch:", i+1, "/",len(dataloader), "Running Loss:",running_loss, "Running Accuracy:",running_acc)
            
    epoch_loss = running_loss
    epoch_acc = running_acc
    return epoch_loss, epoch_acc

def test(dataloader, model, loss_fn):
    model.eval()
    with torch.no_grad():
        for i, imgs in enumerate(dataloader):
            imgs=imgs.to(device)
            pred=model(imgs)
            pred = torch.argmax(pred, dim=1).type(torch.int).cpu()
            print(pred.numpy())
            
            

model = prac4()
model = model.to(device)
loss = nn.CrossEntropyLoss()
lr = 0.001
optimizer=torch.optim.Adam(params=model.parameters(), lr=lr)
epochs=10

for i in range(epochs):
    print("Epoch:", i+1, '/', epochs)
    epoch_loss, epoch_acc = train(trainDL, model, loss, optimizer)
    print("Training:", "Epoch Loss:", epoch_loss, "Epoch Accuracy:", epoch_acc)
    print("--------------------------------------------------")

test(testDL, model, loss)

cuda:0
Epoch: 1 / 10
Batch: 1 / 657 Running Loss: 2.49 Running Accuracy: 18.75
Batch: 101 / 657 Running Loss: 0.68 Running Accuracy: 89.51
Batch: 201 / 657 Running Loss: 0.56 Running Accuracy: 93.01
Batch: 301 / 657 Running Loss: 0.49 Running Accuracy: 94.36
Batch: 401 / 657 Running Loss: 0.44 Running Accuracy: 95.14
Batch: 501 / 657 Running Loss: 0.41 Running Accuracy: 95.66
Batch: 601 / 657 Running Loss: 0.38 Running Accuracy: 96.05
Training: Epoch Loss: 0.37 Epoch Accuracy: 96.23
--------------------------------------------------
Epoch: 2 / 10
Batch: 1 / 657 Running Loss: 0.2 Running Accuracy: 98.44
Batch: 101 / 657 Running Loss: 0.2 Running Accuracy: 98.22
Batch: 201 / 657 Running Loss: 0.18 Running Accuracy: 98.52
Batch: 301 / 657 Running Loss: 0.17 Running Accuracy: 98.49
Batch: 401 / 657 Running Loss: 0.17 Running Accuracy: 98.57
Batch: 501 / 657 Running Loss: 0.16 Running Accuracy: 98.6
Batch: 601 / 657 Running Loss: 0.15 Running Accuracy: 98.61
Training: Epoch Loss: 0.15 Epoch